In [33]:
import pandas as pd
import numpy as np
import ensemble_trading_strategy as ets
import indicators as ind
import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
df = pd.read_csv('../data/S & P 50/AAPL.csv')
df.head()

,date,open,high,low,close,volume
0,2018-01-02 00:00:00.000000,42.540001,43.075001,42.314999,43.064999,102223600.0
1,2018-01-03 00:00:00.000000,43.132500,43.637501,42.990002,43.057499,118071600.0
2,2018-01-04 00:00:00.000000,43.134998,43.367500,43.020000,43.257500,89738400.0
3,2018-01-05 00:00:00.000000,43.360001,43.842499,43.262501,43.750000,94640000.0
4,2018-01-08 00:00:00.000000,43.587502,43.902500,43.482498,43.587502,82271200.0


In [35]:
featured_df = ets.add_features(df)
featured_df.head(10)

,open,high,low,close,volume,sma_20,sma_90,ema_20,ema_90,rsi_16,cci_16,atr,STOCHk_14_3_3,STOCHd_14_3_3,STOCHRSIk_16_14_3_3,STOCHRSId_16_14_3_3,wpr
89,46.935001,47.592499,46.912498,47.509998,111957200.0,43.547250,43.125528,44.179081,43.125528,70.143131,125.338009,0.997764,97.250943,95.364552,100.000000,100.000000,-1.109623
90,47.372501,47.514999,46.862499,47.147499,104848800.0,43.720500,43.170889,44.461787,43.213923,67.519595,101.477234,0.976185,97.252911,96.343498,97.140989,99.046996,-5.985203
91,47.252499,47.382500,46.965000,47.037498,83115200.0,43.874625,43.215111,44.707093,43.297958,66.711947,88.146319,0.941267,95.146824,96.550226,93.399712,96.846900,-7.464701
92,46.695000,46.767502,46.275002,46.610001,94780800.0,43.977125,43.252361,44.888322,43.370750,63.560242,64.009363,0.930094,91.111864,94.503866,86.214321,92.251674,-13.214506
93,46.517502,47.115002,46.500000,47.044998,76732400.0,44.106375,43.288972,45.093720,43.451502,65.337628,65.431677,0.910401,90.652321,92.303670,83.717828,87.777287,-7.363830
94,47.000000,47.227501,46.590000,46.747501,69176000.0,44.283750,43.324083,45.251223,43.523942,63.092664,58.469638,0.893344,89.191507,90.318564,79.640037,83.190729,-11.365134
95,46.797501,46.952499,46.532501,46.577499,73190800.0,44.541125,43.357361,45.377535,43.591053,61.798398,48.359741,0.863760,88.204577,89.349469,77.585242,80.314369,-13.651644
96,47.000000,47.317501,46.727501,46.907501,73603200.0,44.821000,43.394417,45.523246,43.663942,63.354907,55.641817,0.856025,85.147235,87.514440,73.811263,77.012181,-9.603892
97,47.095001,47.220001,46.695000,46.790001,60962800.0,45.123750,43.427417,45.643890,43.732647,62.389451,48.749481,0.835336,82.379411,85.243741,68.105760,73.167422,-12.788815
98,46.587502,47.125000,46.439999,47.090000,80233600.0,45.432625,43.458722,45.781614,43.806435,63.888142,45.206727,0.825940,82.279386,83.268677,57.683620,66.533547,-12.130329


In [36]:
featured_df['prev_3d_price'] = df.close.shift(3)
featured_df['fwd_3d_price'] = df.close.shift(-3)
featured_df.dropna(inplace=True)
featured_df['prev_rets'] = featured_df['prev_3d_price'].pct_change().mul(100)
featured_df['curr_rets'] = featured_df.close.pct_change().mul(100)
featured_df['fwd_rets'] = featured_df['fwd_3d_price'].pct_change().mul(100)

In [37]:
featured_df[['prev_3d_price','close','fwd_3d_price','prev_rets','curr_rets','fwd_rets']].head(10)

,prev_3d_price,close,fwd_3d_price,prev_rets,curr_rets,fwd_rets
89,46.290001,47.509998,46.610001,NaN,NaN,NaN
90,46.512501,47.147499,47.044998,0.480665,-0.762996,0.933271
91,46.840000,47.037498,46.747501,0.704110,-0.233312,-0.632366
92,47.509998,46.610001,46.577499,1.430397,-0.908845,-0.363660
93,47.147499,47.044998,46.907501,-0.762996,0.933271,0.708501
94,47.037498,46.747501,46.790001,-0.233312,-0.632366,-0.250494
95,46.610001,46.577499,47.090000,-0.908845,-0.363660,0.641161
96,47.044998,46.907501,47.037498,0.933271,0.708501,-0.111492
97,46.747501,46.790001,47.145000,-0.632366,-0.250494,0.228545
98,46.577499,47.090000,46.974998,-0.363660,0.641161,-0.360594


In [38]:
featured_df['signals'] = np.where((featured_df['fwd_rets'] > 1), 1, 0)
featured_df['signals'] = np.where((featured_df['fwd_rets'] < -1), -1, featured_df['signals'])
featured_df.dropna(inplace=True)

In [39]:
featured_df.head()

,open,high,low,close,volume,sma_20,sma_90,ema_20,ema_90,rsi_16,...,STOCHd_14_3_3,STOCHRSIk_16_14_3_3,STOCHRSId_16_14_3_3,wpr,prev_3d_price,fwd_3d_price,prev_rets,curr_rets,fwd_rets,signals
90,47.372501,47.514999,46.862499,47.147499,104848800.0,43.720500,43.170889,44.461787,43.213923,67.519595,...,96.343498,97.140989,99.046996,-5.985203,46.512501,47.044998,0.480665,-0.762996,0.933271,0
91,47.252499,47.382500,46.965000,47.037498,83115200.0,43.874625,43.215111,44.707093,43.297958,66.711947,...,96.550226,93.399712,96.846900,-7.464701,46.840000,46.747501,0.704110,-0.233312,-0.632366,0
92,46.695000,46.767502,46.275002,46.610001,94780800.0,43.977125,43.252361,44.888322,43.370750,63.560242,...,94.503866,86.214321,92.251674,-13.214506,47.509998,46.577499,1.430397,-0.908845,-0.363660,0
93,46.517502,47.115002,46.500000,47.044998,76732400.0,44.106375,43.288972,45.093720,43.451502,65.337628,...,92.303670,83.717828,87.777287,-7.363830,47.147499,46.907501,-0.762996,0.933271,0.708501,0
94,47.000000,47.227501,46.590000,46.747501,69176000.0,44.283750,43.324083,45.251223,43.523942,63.092664,...,90.318564,79.640037,83.190729,-11.365134,47.037498,46.790001,-0.233312,-0.632366,-0.250494,0


In [40]:
featured_df.signals.value_counts()

signals
 0    548
 1    343
-1    275
Name: count, dtype: int64

In [41]:
X = featured_df.drop(['prev_3d_price','fwd_3d_price','prev_rets','curr_rets','fwd_rets','signals'],axis=1)
X.head()

,open,high,low,close,volume,sma_20,sma_90,ema_20,ema_90,rsi_16,cci_16,atr,STOCHk_14_3_3,STOCHd_14_3_3,STOCHRSIk_16_14_3_3,STOCHRSId_16_14_3_3,wpr
90,47.372501,47.514999,46.862499,47.147499,104848800.0,43.720500,43.170889,44.461787,43.213923,67.519595,101.477234,0.976185,97.252911,96.343498,97.140989,99.046996,-5.985203
91,47.252499,47.382500,46.965000,47.037498,83115200.0,43.874625,43.215111,44.707093,43.297958,66.711947,88.146319,0.941267,95.146824,96.550226,93.399712,96.846900,-7.464701
92,46.695000,46.767502,46.275002,46.610001,94780800.0,43.977125,43.252361,44.888322,43.370750,63.560242,64.009363,0.930094,91.111864,94.503866,86.214321,92.251674,-13.214506
93,46.517502,47.115002,46.500000,47.044998,76732400.0,44.106375,43.288972,45.093720,43.451502,65.337628,65.431677,0.910401,90.652321,92.303670,83.717828,87.777287,-7.363830
94,47.000000,47.227501,46.590000,46.747501,69176000.0,44.283750,43.324083,45.251223,43.523942,63.092664,58.469638,0.893344,89.191507,90.318564,79.640037,83.190729,-11.365134


In [42]:
y = featured_df['signals']

In [43]:
y.value_counts()

signals
 0    548
 1    343
-1    275
Name: count, dtype: int64

In [44]:
X_scaled = ets.data_preprocess(X)

In [45]:
ets.train_rfc_model(X_scaled, y)

Accuracy of RFC model: 0.3972602739726027


In [46]:
ets.train_bag_model_with_lr(X_scaled, y)

Accuracy of Bagging with LR: 0.4589041095890411


In [54]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Instantiate an XGBoost regressor
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

# Train the model
xg_reg.fit(X_train,y_train)

# Make predictions on the test data
y_pred = xg_reg.predict(X_test)


# Evaluate the model
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("RMSE:", rmse)

RMSE: 0.7065237516115573


In [55]:
from joblib import dump
dump(xg_reg, 'xgboost_model.joblib')

['xgboost_model.joblib']